In [193]:
!pip install torch


In [194]:
import torch
import random
import numpy as np
num_entries= 5000


In [5]:
# now create 5000 databases with 1 person missing 

tensor([1, 0], dtype=torch.uint8)

In [58]:
# well from 1..5000
# copy db and remove an entry at a given place 


In [59]:
def create_db_and_parallels(num_entries):
    db = torch.rand(num_entries) > 0.5
    catalog = []
    for i in range(0,num_entries):
        tmp = torch.cat([db[0:i], db[i+1:]])
        catalog.append(tmp)
    return db , catalog
    

In [60]:
(db, parallels) = create_db_and_parallels(5000)
len(parallels)

len(parallels[0])


4999

In [61]:
# empirically evaluate the sensitivity of a function to the removal of 1 of the db records
# the simplest query we can have is a simple sum 


In [63]:
def sum_entries(db):
    return db.sum()
    
# what is the value of this function on the original db
full_db_result = sum_entries(db)
print ("full db result", full_db_result)

sensitivity = 0
    
for p in parallels: 
    distance = torch.abs(sum_entries(p)-full_db_result)
    if(distance > sensitivity):
        sensitivity = distance
        
sensitivity    

full db result tensor(2491)


tensor(1)

In [98]:
# empirically measuring sensitivity
def sensitivity(query, num_entries=1000):
    (db, parallels) = create_db_and_parallels(num_entries)
    sensitivity = 0
    full_db_result = query(db)
    print ("full db result",full_db_result)
    for p in parallels: 
        distance = torch.abs(query(p)- full_db_result)
        
        if(distance > sensitivity):
            sensitivity = distance
        
    return sensitivity  

In [99]:
query = sum_entries
    
sensitivity(sum_entries,5000)

full db result tensor(2513)


tensor(1)

In [104]:
def query(db):
    return db.float().mean()
    
sensitivity(query,5000)

full db result tensor(0.4902)


tensor(0.0001)

In [115]:
# computes sum over database and returns whether that sum is less (or more)
# than a certain threshold
def is_greater_than_threshold(db,t=5):
    return (db.sum() > t).float()

# create 10 databases
def sensitivity_for_each(query, num_entries=10):
    (db, parallels) = create_db_and_parallels(num_entries)
    sensitivity = 0
    full_db_result = query(db)
    
    print ("full db result",full_db_result,db)
    
    for p in parallels: 
        distance = torch.abs(query(p)- full_db_result)
        print ("distance", distance)

        if(distance > sensitivity):
            sensitivity = distance
    return sensitivity  


sensitivity_for_each(is_greater_than_threshold,10)

full db result tensor(1.) tensor([0, 0, 1, 1, 1, 1, 0, 0, 1, 1], dtype=torch.uint8)
distance tensor(0.)
distance tensor(0.)
distance tensor(1.)
distance tensor(1.)
distance tensor(1.)
distance tensor(1.)
distance tensor(0.)
distance tensor(0.)
distance tensor(1.)
distance tensor(1.)


tensor(1.)

In [116]:
# perform an attack on row 10, can we know of value of row if we remove it?

In [117]:
(db, parallels) = create_db_and_parallels(100)

In [118]:
# remove index 10

In [119]:
pdb = torch.cat([db[0:9], db[10:]])

In [120]:
pdb[10]


tensor(1, dtype=torch.uint8)

In [121]:
db[10]


tensor(0, dtype=torch.uint8)

In [122]:
#differencing attack using addition
sum(db)- sum(pdb)

tensor(1, dtype=torch.uint8)

In [123]:
#differencing attack using mean > 0 means row is a "1" as we just have ones and zeros
sum(db).float()/len(db) - sum(pdb).float()/len(pdb)

tensor(0.0043)

In [177]:
# create db and a copy of it for which every entry has been flipped according to plausible deniability
# if ramndom(0,1) == 1 leave entry
# else if random(0,1) == 1 set entry to 1
#       else set entry to 0

# so 50% of values of original db are left alone, 50% are randomly flipped
import random

def create_augmented_db(num_entries):
    db = torch.rand(num_entries) > 0.5
    # now create copy 
    augmented_database = db.clone();
    for i in range(0,num_entries):
        flip_1 = random.randint(0,1)
        flip_2 = random.randint(0,1)
        if (flip_1 == 0):
            augmented_database[i] = flip_2
            
    # returns original db and teh augmented one
    return (db, augmented_database)


(db, augmented_db) = create_augmented_db(100)

def create_augmented_db(num_entries):
    db = torch.rand(num_entries) > 0.5
    # now create copy 
    augmented_database = db.clone();
    for i in range(0,num_entries):
        flip_1 = random.randint(0,1)
        flip_2 = random.randint(0,1)
        # if 1st coin flip is a 1 nothing happens, things flow as usual
        # if 1st coing flip is a zero we change the result of the entry 
        # to the result of the 2nd coin flip
        if (flip_1 == 0):
            augmented_database[i] = flip_2
            
    # returns original db and teh augmented one
    return (db, augmented_database)




# the noise parameter changes how likely it is for the 1st coin flip to be a 1 versus a zero
# a percentage
def create_augmented_db_with_variable_noise(num_entries, noise):
    db = torch.rand(num_entries) > 0.5
    # now create copy 
    augmented_database = db.clone();
    
    for i in range(0,num_entries):
        flip_1 = 0
        # if noise 0.5 we are on the 'even' case in which 0 and 1 are equally likely
        if (random.uniform(0,1) > noise):
            flip_1 = 1
        
        flip_2 = random.randint(0,1)
        # if 1st coin flip is a 1 nothing happens, things flow as usual
        # if 1st coing flip is a zero we change the result of the entry 
        # to the result of the 2nd coin flip
        if (flip_1 == 0):
            augmented_database[i] = flip_2
            
    # returns original db and teh augmented one
    return (db, augmented_database)


# LOCAL DIFFERENTIAL PRIVACY, ADING NOISE TO INDIVIDUAL DATA POINTS

def query(num_entries, noise = 0.5):
    (db, augmented_db) = create_augmented_db_with_variable_noise(num_entries, noise)
    true_result = torch.mean(db.float())
    
    #augmented result
    ag_result = torch.mean(augmented_db.float())
    # the augmented result is the result of averaging real result with 0.5
    # output of our query on ag_resul is skewed but that is not what we were after
    # we need to de-skew ag_result
    estimated_result = ag_result/noise -((1-noise)*0.5/noise)
    print ("true result  and estimated result ", true_result, estimated_result)
    
        

In [184]:

query(100000,0.9)

true result  and estimated result  tensor(0.4972) tensor(0.4997)


In [181]:
# the more datapoints that we have the more the noise will tend to diffuse

In [185]:
# laplacian mechanism to perform 2 kinds of queries
# sum and mean 
epsilon = 0.5

# remember, this removes an entry at a given place
(db, parallels) = create_db_and_parallels(100)

sum(db)

tensor(49, dtype=torch.uint8)

In [188]:
db.float().mean()

tensor(0.4900)

In [195]:
def sum_query(db):
    return db.sum()

#adding laplacian noise 
# we know in our dB (of 0s and 1s sensitivity of sum is 1)
def laplacian_mechanism(db, query, sensitivity):
    beta = sensitivity/epsilon
    noise = torch.tensor(np.random.laplace(0,beta,1))
    return query(db) + noise
    
    

In [203]:
laplacian_mechanism(db,sum_query,1 )



tensor([46.8743], dtype=torch.float64)

In [204]:
sum_query(db)

tensor(49)

In [213]:
def mean_query(db):
    return torch.mean(db.float())

In [214]:
mean_query(db)

tensor(0.4900)

In [220]:
#sensitivity of sum is 1
laplacian_mechanism(db, sum_query,1)



tensor([7010.2391], dtype=torch.float64)

In [221]:
# assuming sensitivity of mean is 1/100 (if db size is 100)
laplacian_mechanism(db,mean_query,1/100)

tensor([-21.0346], dtype=torch.float64)

In [222]:
# lower values of epsilon will add TOO MUCH noise

In [225]:
epsilon =0.1

In [226]:
laplacian_mechanism(db,mean_query,1/100)

tensor([0.6343], dtype=torch.float64)